In [1]:
import os
import errno

import traceback
import json

import numpy as np

import deepcell
import matplotlib.pyplot as plt

from deepcell.datasets.tracked import hek293,nih_3t3_bench,nih_3t3,hek293_bench,hela_s3_bench,raw2647_bench
from deepcell.datasets.cytoplasm import hela,nih_3t3,cho
from PIL import Image
import imageio
import cv2
import random

import imgaug.augmenters as iaa

import torch
import torchvision.transforms.functional as TF

from skimage import io
import tifffile as tiff

from sklearn.cluster import KMeans

from skimage.transform import resize
from deepcell_tracking.trk_io import load_trks

2023-08-17 17:30:45.239630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mo/anaconda3/envs/cell/lib/python3.9/site-packages/cv2/../../../../lib:
2023-08-17 17:30:45.239661: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/mo/anaconda3/envs/cell/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/mo/anaconda3/envs/cell/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:

def ctc_2_CNet_track(path_X,suff_X,path_Y,suff_Y,start_ind,prompt='hela_ctc',alt=0,save_dir_org='/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells_track/',test=False,step=1,specific_i=None):
    
    #data=[]
    test_entries = []
    train_entries = []
    l_ind=start_ind
    if specific_i is not None:
        end=1
    else:
        end=350
        
    for i in range(end):
        test=False
        save_dir=save_dir_org
        
        
        try:
            
            ind=i+step
            path_r_X0 = path_X + suff_X + '{:03d}'.format(ind) + '.tif'
            path_r_Y0 = path_Y + suff_Y + '{:03d}'.format(ind) + '.tif'
            path_r_X1 = path_X + suff_X + '{:03d}'.format(i) + '.tif'
            path_r_Y1 = path_Y + suff_Y + '{:03d}'.format(i) + '.tif'
            
            if specific_i is not None:
            
                path_r_X0 = path_X + suff_X + '{:03d}'.format(specific_i+1) + '.tif'
                path_r_Y0 = path_Y + suff_Y + '{:03d}'.format(specific_i+1) + '.tif'
                path_r_X1 = path_X + suff_X + '{:03d}'.format(specific_i) + '.tif'
                path_r_Y1 = path_Y + suff_Y + '{:03d}'.format(specific_i) + '.tif'
              
                
                
                

            img_r_X0 = tiff.imread(path_r_X0)
            img_r_Y0 = tiff.imread(path_r_Y0)
            img_r_X1 = tiff.imread(path_r_X1)
            img_r_Y1 = tiff.imread(path_r_Y1)

            max_X = np.max(img_r_X0)
            img_r_X0 = (img_r_X0/max_X)*255
            img_r_X1 = (img_r_X1/max_X)*255
            
         

            
         
            start_x = np.random.randint(0, img_r_X0.shape[1] - 512)
            end_x = start_x + 512
            start_y = np.random.randint(0, img_r_X0.shape[0] - 512)
            end_y = start_y + 512

            img_r_X0 = img_r_X0[start_y:end_y, start_x:end_x]
            img_r_X1 = img_r_X1[start_y:end_y, start_x:end_x]
            img_r_Y0 = img_r_Y0[start_y:end_y, start_x:end_x]
            img_r_Y1 = img_r_Y1[start_y:end_y, start_x:end_x]

            
            probability = 0.1  # 10% probability

            # Generate a random number between 0 and 1
            random_number = random.random()

            # Check if the random number is less than the desired probability
            if random_number < probability:
                test=True
                #print('is_True',)
                
            if test:
                save_dir='/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells_track_rd/'
                
            
            img_r_Y1=connect_matching_dots(img_r_Y0,img_r_Y1,path_Y,ind, img_r_X0)

            fig, ax = plt.subplots()
            im0 = ax.imshow(img_r_Y0, cmap='viridis')
            fig.colorbar(im0, ax=ax)
            fig.canvas.draw()
            combined_imag0 = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            combined_imag0 = combined_imag0.reshape(fig.canvas.get_width_height()[::-1] + (3,))

            fig, ax = plt.subplots()
            im1 = ax.imshow(img_r_Y1, cmap='viridis')
            fig.colorbar(im1, ax=ax)
            fig.canvas.draw()
            combined_imag1 = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
            combined_imag1 = combined_imag1.reshape(fig.canvas.get_width_height()[::-1] + (3,))

            combined_imag0 = (combined_imag0 * 255).astype(np.uint8)
            combined_imag1 = (combined_imag1 * 255).astype(np.uint8)
            
            j=random.randint(1, 10000)

            output_path_X0 = save_dir + 'target/' + str(start_ind + i + 1)+'_'+ str(i)+ '.png'
            output_path_Y0 = save_dir + 'source/' + str(start_ind + i + 1) +'_'+ str(i)+ '.png'
            output_path_X1 = save_dir + 'target/' + str(start_ind + i) + '_'+ str(i)+'.png'
            output_path_Y1 = save_dir + 'source/' + str(start_ind + i) + '_'+ str(i)+'.png'
            
            
            img_r_Y1 = (img_r_Y1).astype(np.uint8)
            
            
            img_r_X1,img_r_Y1=augment_images(img_r_X1, img_r_Y1, alt)
            
            #cv2.imwrite(output_path_Y0, combined_imag0)
            cv2.imwrite(output_path_Y1, img_r_Y1)
            #cv2.imwrite(output_path_X0, img_r_X0)
            cv2.imwrite(output_path_X1, img_r_X1)
            l_ind = l_ind + 1
        
    

            

            entry = {
            "source": 'source/' + str(start_ind + i) + '_'+ str(i)+'.png',
            "target": 'target/' + str(start_ind + i) + '_'+ str(i)+'.png',
            "prompt": prompt
            }

            # Append the entry to the data list
            #data.append(entry)
            
            if test:
                test_entries.append(entry)
                
            else:
                train_entries.append(entry)
                

            

                # Write the data list as JSON to a file



        except Exception as e:
            if not str(e).startswith("[Errno 2]"):
                 traceback.print_exc()
                    #print("An error occurred:", str(e))

    save_list=['/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells_track/','/home/mo/Desktop/IWR/Cell_GT_Proj/CNet_cells_track_rd/']    
    for h in range(2): 
        save_dir=save_list[h]
        if h==0:
            data = train_entries
        else:
            data = test_entries
    
        output_path = save_dir+"/prompt.json"    
        existing_data = []



        # Read existing data from the JSON file, if it exists
        try:
            with open(output_path, "r") as infile:
                for line in infile:
                    line = line.strip()  # Remove leading/trailing whitespaces
                    if line:
                        data_j = json.loads(line)
                        existing_data.append(data_j)
        except FileNotFoundError:
            pass

        #print('!!!!!!ex',existing_data)
        #print('!!!!!!dat',data)
        # Append new entries to the existing data
        existing_data=existing_data+data
        #print('!!!!!!ex2',existing_data)

        # Write the updated data to the JSON file
        with open(output_path, "w") as outfile:
            for entry in existing_data:
                #print('ent',entry)
                json.dump(entry, outfile)
                outfile.write('\n') 

    return l_ind
        



def augment_images(image_X, image_Y, alt):

    # Rotate the images based on the 'alt' value
    rotate_degrees = alt * 90  # Rotate by 90, 180, 270, or 360 degrees
    aug = iaa.Affine(rotate=rotate_degrees)
    augmented_images = aug(images=[image_X, image_Y])
    
    # Return the augmented images
    augmented_image_X, augmented_image_Y = augmented_images
    return augmented_image_X, augmented_image_Y



def generate_similar_positions(positions,num):
    positions = np.array(positions)
    # Step 1: Calculate pairwise distances
    pairwise_distances = np.linalg.norm(positions[:, None] - positions, axis=2)

    # Step 2: Create an empty list for new positions
    new_positions = []
    
    n_clusters=int(np.sqrt(len(positions)/2))
    n_clusters=int(len(positions)/2)

    # Step 3: Generate random distances with the same distribution
    random_distances = np.random.choice(pairwise_distances.flatten(), size=len(positions)-1)

    # Step 4: Apply KMeans clustering to the positions
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(positions)
    cluster_centers = kmeans.cluster_centers_

    # Step 5-11: Generate new positions
    for random_distance in random_distances:
        # Step 6: Randomly select two cluster centers
        idx1, idx2 = np.random.choice(len(cluster_centers), size=2, replace=False)
        p1, p2 = cluster_centers[idx1], cluster_centers[idx2]

        # Step 7-9: Calculate displacement vector and new position
        displacement = (p2 - p1) / np.linalg.norm(p2 - p1)
        new_position = p1 + displacement * random_distance

        # Step 10: Append new position to the list
        new_positions.append(new_position)

    # Step 11: Convert the list to numpy array
    new_positions = np.array(new_positions)
    
    if num != None and len(new_positions)-num > 2:
        #print('nn1',new_positions,num)
        #new_positions = random.sample(new_positions, len(new_positions))
        #print('nn2',new_positions,num)
        new_positions=new_positions[:num]
        

    return new_positions


def random_number():
    if random.random() < 0.25:  # Random float:  0.0 <= x < 1.0
        return True, random.randint(1, 6)  # Random int: 1 <= x <= 6
    else:
        return False, None

'''
def connect_matching_dots(img1, img2, path, t, cells, test=False,deep=False,lineage=None):
    # Find unique pixel values in both images
    unique_values_1 = np.unique(img1)
    unique_values_2 = np.unique(img2)
    
    #print('cmd',img1.shape,img2.shape,cells.shape)

    # Create an output image for dots and lines separately
    output_img_dots = np.zeros_like(img1)
    output_img_lines = np.zeros_like(img1)
    output_img_splits = np.zeros_like(img1)

    # Initialize 3-channel image with zeros
    output_img_dots = np.stack((output_img_dots,)*3, axis=-1)
    output_img_lines = np.stack((output_img_lines,)*3, axis=-1)
    output_img_splits = np.stack((np.zeros_like(img1),)*3, axis=-1)

    # Draw dots in blue
    output_img_dots[:, :, 2] = cells  # RGB color for Blue 

    # Draw lines and circles in green
    for value in unique_values_1[unique_values_1 > 0]:
        # Find the coordinates of matching dots in both images
        coordinates_img1 = np.argwhere(img1 == value)
        coordinates_img2 = np.argwhere(img2 == value)

        if coordinates_img2.size > 0: 
            coord1 = np.mean(coordinates_img1, axis=0, dtype=int)
            coord2 = np.mean(coordinates_img2, axis=0, dtype=int)
            if test:
                coord2 = test_transform(coord1, coord2)
            cv2.line(output_img_lines, tuple(coord1[::-1]), tuple(coord2[::-1]), color=[55, 200, 0], thickness=3)
            cv2.circle(output_img_lines, tuple(coord2[::-1]), 8, color=[0, 255, 0], thickness=-1)  # Filled circle
        else:
            if deep:
                cs = check_split_deep(path, t, value,lineage)
            else:
                cs = check_split(path, t, value)
                
            #print('cs', cs)
            if cs[0]:
                coordinates_img1 = np.argwhere(img1 == value)
                coordinates_img2 = np.argwhere(img2 == cs[1])
                if coordinates_img2.size > 0: 
                    coord1 = np.mean(coordinates_img1, axis=0, dtype=int)
                    coord2 = np.mean(coordinates_img2, axis=0, dtype=int)
                    
                    cv2.line(output_img_splits, tuple(coord1[::-1]), tuple(coord2[::-1]), color=[200, 55, 0], thickness=3)
                    cv2.circle(output_img_splits, tuple(coord2[::-1]), 8, color=[255, 0, 0], thickness=-1)  # Filled circle

    # Stack images along the channel dimension: Red for splits, Green for lines, Blue for dots
    output_img = output_img_dots + output_img_lines + output_img_splits
    return output_img

'''


def connect_matching_dots(img1, img2, path, t, cells, frame,test=False, deep=False, lineage=None):
    # Find unique pixel values and their counts in both images
    unique_values_1, counts_1 = np.unique(img1, return_counts=True)
    unique_values_2, counts_2 = np.unique(img2, return_counts=True)
    counts_2_dict = dict(zip(unique_values_2, counts_2))
    
    # Create an output image for dots and lines separately
    output_img_dots = np.zeros_like(img1)
    output_img_lines = np.zeros_like(img1)
    output_img_splits = np.zeros_like(img1)

    # Initialize 3-channel image with zeros
    output_img_dots = np.stack((output_img_dots,)*3, axis=-1)
    output_img_lines = np.stack((output_img_lines,)*3, axis=-1)
    output_img_splits = np.stack((np.zeros_like(img1),)*3, axis=-1)

    # Draw dots in blue
    output_img_dots[:, :, 2] = cells  

    # Draw lines and circles in green
    for value in unique_values_1[unique_values_1 > 0]:
        # Find the coordinates of matching dots in both images
        coordinates_img1 = np.argwhere(img1 == value)
        coordinates_img2 = np.argwhere(img2 == value)

        if coordinates_img2.size > 0: 
            
            
            coord1 = np.mean(coordinates_img1, axis=0, dtype=int)
            coord2 = np.mean(coordinates_img2, axis=0, dtype=int)
            
            flag=check_split_timeframe(frame, value, lineage[t])

            # Define the radius for this specific point
            if value in counts_2_dict:
                radius = int(map_value_linear(counts_2_dict[value],10, 1500, 2, 8))
            else:
                radius = 8  # Default radius if not in dictionary
            
            if test:
                coord2 = test_transform(coord1, coord2)
            
            if flag==0:
                cv2.line(output_img_lines, tuple(coord1[::-1]), tuple(coord2[::-1]), color=[55, 200, 0], thickness=3)
                cv2.circle(output_img_lines, tuple(coord2[::-1]), radius, color=[0, 255, 0], thickness=-1) # Filled circle
            elif flag==1:
                cv2.line(output_img_lines, tuple(coord1[::-1]), tuple(coord2[::-1]), color=[100, 255, 0], thickness=3)
                cv2.circle(output_img_lines, tuple(coord2[::-1]), radius, color=[255, 255, 0], thickness=-1) # Filled circle
            elif flag==2:
                #print('flag2',t,frame)
                cv2.line(output_img_lines, tuple(coord1[::-1]), tuple(coord2[::-1]), color=[55, 200, 0], thickness=3)
                cv2.circle(output_img_lines, tuple(coord2[::-1]), radius, color=[255, 255, 0], thickness=-1) # Filled 
                
                
           
        else:
            if deep:
                cs = check_split_deep(path, t, value, lineage)
            else:
                cs = check_split(path, t, value)
                
            if cs[0]:
                coordinates_img1 = np.argwhere(img1 == value)
                coordinates_img2 = np.argwhere(img2 == cs[1])
                if coordinates_img2.size > 0: 
                    coord1 = np.mean(coordinates_img1, axis=0, dtype=int)
                    coord2 = np.mean(coordinates_img2, axis=0, dtype=int)
                    cv2.line(output_img_splits, tuple(coord1[::-1]), tuple(coord2[::-1]), color=[200, 55, 0], thickness=3)
                    cv2.circle(output_img_splits, tuple(coord2[::-1]), radius, color=[255, 0, 0], thickness=-1)  # Filled circle

    # Stack images along the channel dimension: Red for splits, Green for lines, Blue for dots
    output_img = output_img_dots + output_img_lines + output_img_splits
    return output_img






def check_split(path,t,value):
    data = np.loadtxt(path+'man_track.txt')

    # Split the data into columns
    start = data[:, 1]
    ide = data[:, 0][start==t]
    
    end = data[:, 2][start==t]
    parent = data[:, 3][start==t]
    
    try:
        par = parent[ide==value]
        print('p_t_pa_v',par,t,path,value)
        
        if len(ide[parent==par])==2:
            print('ide',ide[parent==par])
            return True,par[0]
        else:
            return False,0
        
    except:
        return False,0
    
    
def check_split_deep(path,j,value,lineage):
    
    par=lineage[j][value]['parent']
    
    if par != None:
        #print('csd',j,value,par)
        return True,par
    else:
        return False,0
        
        
    
    
    
    
    
    
def test_transform(coord1, coord2_mean):
    distance = np.linalg.norm(coord2_mean - coord1)
    scale_factor = 3  # Distance scale factor

    if distance > 0:
        # Generate a random orientation
        angle = random.uniform(0, 2 * np.pi)

        # Calculate the new coordinate
        direction = (coord2_mean - coord1) / distance
        offset = direction * scale_factor * distance
        offset_rotated = np.array([np.cos(angle), -np.sin(angle), np.sin(angle), np.cos(angle)]).reshape((2, 2)).dot(offset)
        coord2 = coord2_mean + offset_rotated.astype(int)
    else:
        # If the distance is 0, set coord2 to coord2_mean
        coord2 = coord2_mean
    return coord2

def deep_2_CNet_track(X,Y,lineage,start_ind,prompt='hela_ctc',alt=0,save_dir_org='/media/mo/Label/CNet_deepcell_track/',test=False,step=1,specific_i=None):
    test_entries = []
    train_entries = []
    data=[]
    l_ind=start_ind
    if test:
        end_j= 1
    else:
        end_j= X.shape[0]
        
    
    for j in range(X.shape[0]):
        for i in range(X.shape[1]-step):
            test=False
            save_dir=save_dir_org
            
            frame_X0 = X[j, i+step, :, :, :]
            frame_X1 = X[j, i, :, :, :]
            frame_Y0 = Y[j, i+step, :, :, :]
            frame_Y1 = Y[j, i, :, :, :]
            if np.mean(frame_X0) != 0:

                #print('mean_j_i',j,i, np.mean(frame_X))
                #output_path_Y_c = save_dir+'source/'+str(start_ind+i)+'contr.png'

                max_X=np.max(X[j,...])
                frame_X0 = (frame_X0/max_X)*255
                frame_X1 = (frame_X1/max_X)*255
                #frame_Y = (frame_Y/max_X)*255


                #print('shape',frame_X.shape)
                #start_x = np.random.randint(0, frame_X.shape[0] - 512)
                start_x = 0
                end_x = start_x + 512
                #start_y = np.random.randint(0, frame_X.shape[1] - 512)
                start_y=0
                end_y = start_y + 512

                # Crop the random region from img_r_X
                img_r_X0 = np.squeeze(frame_X0[start_y:end_y, start_x:end_x])
                img_r_X1 = np.squeeze(frame_X1[start_y:end_y, start_x:end_x])

                # Crop the random region from img_r_Y
                img_r_Y0 = np.squeeze(frame_Y0[start_y:end_y, start_x:end_x])
                img_r_Y1 = np.squeeze(frame_Y1[start_y:end_y, start_x:end_x])
                
                
                probability = 0.1  # 10% probability

                # Generate a random number between 0 and 1
                random_number = random.random()


                
                if random_number < probability:
                      test=True
                
                # Specify the output file path
                if test:
                    save_dir='/media/mo/Label/CNet_deepcell_track_rd/'
                      
                img_r_Y1=connect_matching_dots(img_r_Y0,img_r_Y1,'None',j, img_r_X0, i+step,deep=True,lineage=lineage)
                
                #print('save_dir',save_dir)

                output_path_X1 = save_dir + 'target/' +str(start_ind+i+1)+'_'+str(j)+'.png'
                output_path_Y1 = save_dir + 'source/' +str(start_ind+i+1)+'_'+str(j)+'.png'


                img_r_X1,img_r_Y1=augment_images(img_r_X1, img_r_Y1, alt)
                
                img_r_Y1 = (img_r_Y1).astype(np.uint8)
                
                

                #cv2.imwrite(output_path_Y0, combined_imag0)
                cv2.imwrite(output_path_Y1, img_r_Y1)
                #cv2.imwrite(output_path_X0, img_r_X0)
                cv2.imwrite(output_path_X1, img_r_X1)
                l_ind = l_ind + 1





                entry = {
                "source": 'source/' + str(start_ind+i+1)+'_'+str(j)+'.png',
                "target": 'target/' + str(start_ind+i+1)+'_'+str(j)+'.png',
                "prompt": prompt
                }

                # Append the entry to the data list
                #data.append(entry)

                if test:
                    test_entries.append(entry)

                else:
                    train_entries.append(entry)


            

                # Write the data list as JSON to a file



      
    save_list=['/media/mo/Label/CNet_deepcell_track/','/media/mo/Label/CNet_deepcell_track_rd/']    
    for h in range(2): 
        save_dir=save_list[h]
        if h==0:
            data = train_entries
        else:
            data = test_entries
    
        output_path = save_dir+"/prompt.json"    
        existing_data = []



        # Read existing data from the JSON file, if it exists
        try:
            with open(output_path, "r") as infile:
                for line in infile:
                    line = line.strip()  # Remove leading/trailing whitespaces
                    if line:
                        data_j = json.loads(line)
                        existing_data.append(data_j)
        except FileNotFoundError:
            pass

        #print('!!!!!!ex',existing_data)
        #print('!!!!!!dat',data)
        # Append new entries to the existing data
        existing_data=existing_data+data
        #print('!!!!!!ex2',existing_data)

        # Write the updated data to the JSON file
        with open(output_path, "w") as outfile:
            for entry in existing_data:
                #print('ent',entry)
                json.dump(entry, outfile)
                outfile.write('\n') 

    return l_ind
    
    
    
    
    
def map_value_linear(value, in_min, in_max, out_min, out_max):
    # Ensure the input value is within the input range
    value = max(min(value, in_max), in_min)
    
    if value==in_min:
        return 1
    elif value== in_max:
        return 8

    # Map the value to the output range using linear interpolation
    return int((value - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)
  
    
    
    
    
    
def check_split_timeframe(t, cell_id, lineage):
    # Loop through each cell in the lineage
    for cell_data in lineage.values():
        # Check if the cell_id is a daughter of the current cell
        if cell_id in cell_data['daughters']:
            # Calculate the difference between the current timeframe and frame_div
            time_difference = t - cell_data['frame_div']
            
            # Check the conditions for flag=1 and flag=2
            if time_difference == 2:
                return 1
            elif time_difference == 3:
                return 2
    
    # If neither condition is met, return flag=0
    return 0

    

In [3]:
print(stop)


NameError: name 'stop' is not defined

In [ ]:
path_X = ['/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/01/']#,
          #'/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/02/']
suff_X = 't' 


path_Y = ['/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/01_GT/TRA/']#
          #'/home/mo/Desktop/IWR/Cell_GT_Proj/CTC_datasets/Fluo-C2DL-Huh7/02_GT/TRA/']
suff_Y='man_track'

prompt_a='hela_ctc, cell, microscopy image, grayscale'
prompt_b='fluo_ctc, cell, tracking, microscopy image, grayscale'

alt=[0,1,2,3]
steps=[1,3,5]
l_ind=0
split_path_0=[3]
split_path_1=[23,17,25,14,26]

for g in range(4):
    for k in range(len(path_X)):
        for q in steps:
            for h in range(len(alt)):
                if k < 2:
                    prompt_ctc=prompt_a
                else:
                    prompt_ctc=prompt_b
                l_ind=ctc_2_CNet_track(path_X[k],suff_X,path_Y[k],suff_Y,start_ind=l_ind,prompt=prompt_b,alt=alt[h],step=q)
            #print(l_ind)


        if k==0:
            split_i=split_path_0
        else:
            split_i=split_path_1

        for t in split_i:
            l_ind=ctc_2_CNet_track(path_X[k],suff_X,path_Y[k],suff_Y,start_ind=l_ind,prompt=prompt_b,alt=alt[h],specific_i=t)


        
        


In [4]:
data_dir = '/media/mo/Label/DynamicNuclearNet-tracking-v1_0/'
data = load_trks(os.path.join(data_dir, 'test.trks'))

X = data['X']
Y = data['y']
lineages = data['lineages']
print(lineages[0])
prompt='cell, microscopy, image'


{1: {'label': 1, 'frames': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 'daughters': [], 'capped': False, 'frame_div': None, 'parent': None}, 2: {'label': 2, 'frames': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 'daughters': [], 'capped': False, 'frame_div': None, 'parent': None}, 3: {'label': 3, 'frames': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 'daughters': [], 'capped': False, 'frame_div': None, 'parent': None}, 4: {'label': 4, 'frames': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 'daughters': [], 'capped': False, 'frame_div'

In [5]:
print(lineages[0][8])

{'label': 8, 'frames': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], 'daughters': [], 'capped': False, 'frame_div': None, 'parent': None}


In [6]:

alt=[0,1,2,3]
#alt=[0]
steps=[1]
l_ind=0
for q in steps:
    for h in range(len(alt)):
        print('!!!!!!!!!!!!!!!!!!!!!!!!',q,h)
        l_ind=deep_2_CNet_track(X,Y,lineages,start_ind=l_ind,prompt=prompt,alt=alt[h],step=q)
            #print(l_ind)





#deep_2_CNet_track(X,Y,lineages,start_ind=0,prompt=prompt,alt=0)


#alt=[0,1,2,3]

#l_ind=0
#for h in range(len(alt)):
#    l_ind=deep_2_CNet(X,y,start_ind=l_ind,prompt=prompt,alt=alt[h])
#    print(l_ind)
        
        
#l_ind=0
#for h in range(len(alt)):
#    l_ind=deep_2_CNet(X,y,start_ind=l_ind,prompt=prompt,alt=alt[h],test=True)
#    print(l_ind)
        

!!!!!!!!!!!!!!!!!!!!!!!! 1 0


In [ ]:
# Path to the image folder
image_folder = "/home/mo/Desktop/IWR/Cell_GT_Proj/image_log/val"

# Create a new folder for the combined images
output_folder = "/home/mo/Desktop/IWR/Cell_GT_Proj/image_log/assembled_val"
os.makedirs(output_folder, exist_ok=True)

# Get a list of all image files in the folder
image_files = [filename for filename in os.listdir(image_folder) if filename.endswith(".png")]

# Process each set of corresponding images
for image_file in image_files:
    # Extract the common identifier from the image filename
    identifier = image_file.split("_gs-", 1)[1].split(".")[0]

    # Construct the filenames of the three corresponding images
    samples_file = os.path.join(image_folder, f"samples_cfg_scale_9.00_gs-{identifier}.png")
    reconstruction_file = os.path.join(image_folder, f"reconstruction_gs-{identifier}.png")
    control_file = os.path.join(image_folder, f"control_gs-{identifier}.png")

    # Open the images
    samples_image = Image.open(samples_file)
    reconstruction_image = Image.open(reconstruction_file)
    control_image = Image.open(control_file)

    # Resize the images by half
    new_size = (samples_image.width // 2, samples_image.height // 2)
    samples_image = samples_image.resize(new_size)
    reconstruction_image = reconstruction_image.resize(new_size)
    control_image = control_image.resize(new_size)

    # Create a new image with the combined images
    combined_image = Image.new("RGB", (new_size[0], new_size[1] * 2))
    combined_image.paste(samples_image, (0, 0))
    #combined_image.paste(reconstruction_image, (0, new_size[1]))
    combined_image.paste(control_image, (0, new_size[1]))

    # Save the combined image in the output folder
    output_file = os.path.join(output_folder, f"combined_{identifier}.png")
    combined_image.save(output_file)

    # Print the saved file path
    print(f"Combined image saved: {output_file}")